## **Step 1: Installations**

In [ ]:
!pip install transformers datasets huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


## **Step 2: Imports**

In [ ]:
from datasets import load_dataset
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

## **Step 3: Data Explorations**

In [ ]:
dataset = load_dataset("ncbi/ncbi_disease")

README.md:   0%|          | 0.00/9.70k [00:00<?, ?B/s]

ncbi_disease.py:   0%|          | 0.00/5.83k [00:00<?, ?B/s]

The repository for ncbi/ncbi_disease contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/ncbi/ncbi_disease.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/5433 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/924 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/941 [00:00<?, ? examples/s]

In [ ]:
# Accessing the data
train_data = dataset['train']
dev_data = dataset['validation']
test_data = dataset['test']

# Example of printing a sample
print(train_data[0])

{'id': '0', 'tokens': ['Identification', 'of', 'APC2', ',', 'a', 'homologue', 'of', 'the', 'adenomatous', 'polyposis', 'coli', 'tumour', 'suppressor', '.'], 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0]}


In [ ]:
pd.DataFrame.from_dict(test_data)

,id,tokens,ner_tags
0,0,"[Clustering, of, missense, mutations, in, the,...","[0, 0, 0, 0, 0, 0, 1, 2, 2, 0, 0, 0, 1, 2, 2, ..."
1,1,"[Ataxia, -, telangiectasia, (, A, -, T, ), is,...","[1, 2, 2, 0, 1, 2, 2, 0, 0, 0, 1, 2, 2, 2, 2, ..."
2,2,"[The, risk, of, cancer, ,, especially, lymphoi...","[0, 0, 0, 1, 0, 0, 1, 2, 0, 0, 0, 0, 0, 1, 2, ..."
3,3,"[By, analysing, tumour, DNA, from, patients, w...","[0, 0, 1, 0, 0, 0, 0, 1, 2, 2, 2, 2, 2, 0, 1, ..."
4,4,"[In, marked, contrast, to, the, ATM, mutation,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0, 0, 0, ..."
...,...,...,...
936,936,"[In, an, attempt, to, resolve, this, issue, ,,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
937,937,"[These, reagents, detect, a, 220, -, kD, prote...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
938,938,"[Immunohistochemical, staining, of, human, bre...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
939,939,"[Conversely, ,, BRCA1, expression, was, reduce...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
print("train_size = {}, dev_size = {}, test_size = {}".format(len(train_data), len(dev_data), len(test_data)))

train_size = 5433, dev_size = 924, test_size = 941


## **Step 4: Model Building**

- **Meaning of Ner_tags labels**
   * 0: No disease mentioned &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;-> O
   * 1: First token of a disease mention&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;-> B-Disease
   * 2: Subsequent tokens of a disease mention &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;-> I-Disease

- **Tasks**
  * Zero-Shot NER on biomedical dataset evaluation with Biomedical LLMs as well as  general
  * Zero-shot with self-improving models - UMLS integration + adaptive consistency/self-consistency & self evaluation using beam search

### **Task 1: Zero-Shot Learning Evaluation**

### **a) Models**
- GLiNER
- Universal-ner

## **Model 1: GLiNER**

In [ ]:
!pip install gliner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 102.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.2 MB/s eta 0:00:00


###**Imports**

In [ ]:
from gliner import GLiNER
import time
from rich.console import Console

### **Model**

In [ ]:
model = GLiNER.from_pretrained("urchade/gliner_base")

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/4.78k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

gliner_config.json:   0%|          | 0.00/732 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/792M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


### **Experimental Playground**

In [ ]:
labels = ["Disease"]

In [ ]:
" ".join(test_data[0]['tokens'])

'Clustering of missense mutations in the ataxia - telangiectasia gene in a sporadic T - cell leukaemia .'

In [ ]:
entities = model.predict_entities(' '.join(test_data[2]['tokens']), labels)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
entities = model.predict_entities(' '.join(test_data[0]['tokens']), labels, threshold=0.2)

print(entities)
for entity in entities:
    print(entity["text"], "=>", entity["label"])

[{'start': 40, 'end': 63, 'text': 'ataxia - telangiectasia', 'label': 'Disease', 'score': 0.20026715099811554}, {'start': 83, 'end': 101, 'text': 'T - cell leukaemia', 'label': 'Disease', 'score': 0.8327548503875732}]
ataxia - telangiectasia => Disease
T - cell leukaemia => Disease


In [ ]:
print(test_data[2]['tokens'])

['The', 'risk', 'of', 'cancer', ',', 'especially', 'lymphoid', 'neoplasias', ',', 'is', 'substantially', 'elevated', 'in', 'A', '-', 'T', 'patients', 'and', 'has', 'long', 'been', 'associated', 'with', 'chromosomal', 'instability', '.']


In [ ]:
test_data[0]['ner_tags']

[0, 0, 0, 0, 0, 0, 1, 2, 2, 0, 0, 0, 1, 2, 2, 2, 2, 0]

In [ ]:
model.predict_entities(' '.join(test_data[0]['tokens']), labels=["Disease"], threshold=0.3)

[{'start': 83,
  'end': 101,
  'text': 'T - cell leukaemia',
  'label': 'Disease',
  'score': 0.8327548503875732}]

In [ ]:
model.predict_entities(' '.join(test_data[0]['tokens']), labels=["Disease"], threshold=0.2)

[{'start': 40,
  'end': 63,
  'text': 'ataxia - telangiectasia',
  'label': 'Disease',
  'score': 0.20026715099811554},
 {'start': 83,
  'end': 101,
  'text': 'T - cell leukaemia',
  'label': 'Disease',
  'score': 0.8327548503875732}]

### **GLiNER baseline evaluation**

In [ ]:
def map_entities_to_labels(tokens, entities):
    labels = [0] * len(tokens)  # Initialize all labels to 0 (non-entity)
    for entity in entities:
        start = tokens.index(entity["text"].split()[0])
        end = start + len(entity["text"].split())
        labels[start] = 1  # B-Disease
        for i in range(start + 1, end):
            labels[i] = 2  # I-Disease
    return labels

In [ ]:
predicted_ner_tags = []

for i in range(len(test_data)):

  # converting token input data to sentence to pass to GLiNER model
  tokens = test_data[i]['tokens']
  text = ' '.join(tokens)

  # zero-shot model prediction
  entities = model.predict_entities(text, labels=["Disease"], threshold=0.2)

  # mapping the prediction to the format available in dataset
  numeric_labels = map_entities_to_labels(tokens, entities)
  predicted_ner_tags.append(numeric_labels)

In [ ]:
df = pd.DataFrame.from_dict(test_data)

gliner_df = df.copy()

gliner_df['gliner_ner_tags_pred'] = predicted_ner_tags

In [ ]:
gliner_df.to_csv('GLiNER_Predictions.csv', index=False)

In [ ]:
def calculate_f1_score(true_labels, pred_labels):

    tp = fp = fn = 0

    for true_seq, pred_seq in zip(true_labels, pred_labels):
        true_entities = extract_entities(true_seq)
        pred_entities = extract_entities(pred_seq)

        true_set = set(true_entities)
        pred_set = set(pred_entities)

        tp += len(true_set & pred_set)
        fp += len(pred_set - true_set)
        fn += len(true_set - pred_set)

    # Calculate precision and recall
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0

    # Calculate F1 score
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return f1_score

In [ ]:
true_labels = gliner_df['ner_tags'].tolist()
pred_labels = gliner_df['gliner_ner_tags_pred'].tolist()

# Calculate F1 score for the "Disease" entity type
f1_disease = calculate_f1_score(true_labels, pred_labels)

print(f"F1 Score for Disease Entity: {f1_disease:.4f}")

F1 Score for Disease Entity: 0.5237


In [ ]:
df.to_csv('NCBI_subset.csv', index=False)